# Set Up to Collect Data

In [1]:
client_key = 'U1u6vGjWc29m9xnPPm8VPfKiT'
client_secret = 'BAGxUP2vaaNgxgs80PAdJxT2ofp2wItRmURMOhIdfR6tiTFIoo'

In [2]:
import requests
import base64

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [3]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}

In [4]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [5]:
json_data =  response.json()

{'token_type': 'bearer',
 'access_token': 'AAAAAAAAAAAAAAAAAAAAAGGTRQEAAAAA%2FmRkYlXBqrl%2FK3nYg%2FbW%2F%2FX0NDE%3DphL8xuqPTvKAsWwvVV4bNab9ve58CYNqzoqgd8IdXeVoRjGMie'}

In [6]:
access_token = json_data['access_token']

# Search for Tweets with #Olympics and Cycling
Here, I have search for tweets with the following link:
https://twitter.com/search?q=%23olympics%20cycling&src=typed_query

In [8]:
max_tweets=10000
tw_limit=100# we can get only 100 tweet per call with standard search api
tweets = []

In [13]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)}

parameters = { 'q': '#olympics AND cycling','result_type': 'recent','count': 100}

search_url = base_url+'1.1/search/tweets.json'

response = requests.get(search_url, headers=search_headers, params=parameters)

print(response.status_code)

200


In [14]:
olympic_cycling = response.json()

In [15]:
len(olympic_cycling['statuses'])

100

In [18]:
tweets +=olympic_cycling['statuses']

In [19]:
 ids = [tw['id'] for tw in olympic_cycling['statuses']]

In [20]:
min_id = min(ids)

In [21]:
tw_ids = []
tw_ids+=ids

As per twitter api documentation we can use

max_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

# Start to gather tweets

In [22]:
for  i in range(max_tweets//tw_limit -1):
    parameters = { 'q': '#olympics AND cycling',
                    'result_type': 'recent',
                'count': tw_limit,
                  'max_id':min_id
                 }
    print("searching tweets with id  < {}".format(min_id))
    search_url = base_url+'1.1/search/tweets.json'
    response = requests.get(search_url, headers=search_headers, params=parameters)
    olympic_cycling = response.json()
    ids = [tw['id'] for tw in olympic_cycling['statuses']]
    tw_ids+=ids
    print(i, len(olympic_cycling['statuses']))
    min_id = min(ids)
    tweets +=olympic_cycling['statuses']

searching tweets with id  < 1421746970360631299
0 100
searching tweets with id  < 1421675650234998785
1 100
searching tweets with id  < 1421581573485387780
2 100
searching tweets with id  < 1421337224130580484
3 100
searching tweets with id  < 1421172971973091328
4 100
searching tweets with id  < 1421070710022213638
5 100
searching tweets with id  < 1421007603526668292
6 100
searching tweets with id  < 1420953653603708928
7 100
searching tweets with id  < 1420862681150590976
8 100
searching tweets with id  < 1420811443587407877
9 100
searching tweets with id  < 1420790392673427461
10 100
searching tweets with id  < 1420772757185470473
11 100
searching tweets with id  < 1420748027111755789
12 100
searching tweets with id  < 1420729327231062019
13 100
searching tweets with id  < 1420706131140304899
14 100
searching tweets with id  < 1420686136964485123
15 100
searching tweets with id  < 1420672044161769473
16 100
searching tweets with id  < 1420658532999905281
17 100
searching tweets wit

In [23]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 6104


In [24]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [25]:
pprint.pprint(tweets[0:max_tweet_to_print])

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Sun Aug 01 16:53:32 +0000 2021',
  'entities': {'hashtags': [{'indices': [0, 9], 'text': 'Olympics'}],
               'symbols': [],
               'urls': [{'display_url': 'twitter.com/i/web/status/1…',
                         'expanded_url': 'https://twitter.com/i/web/status/1421876763240640513',
                         'indices': [117, 140],
                         'url': 'https://t.co/RJo9zFeZiP'}],
               'user_mentions': []},
  'favorite_count': 0,
  'favorited': False,
  'geo': None,
  'id': 1421876763240640513,
  'id_str': '1421876763240640513',
  'in_reply_to_screen_name': None,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'is_quote_status': False,
  'lang': 'en',
  'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
  'place': None,
  'retweet_count': 0,
  'retweeted': False,
  'source': '<a href=

In [26]:
for status in tweets[0:max_tweet_to_print]:
    print(status['text'] + '\n')

#Olympics I don’t want to take anything away from BMX Cycling but that is literally the same shit I was doing on my… https://t.co/RJo9zFeZiP

I am really enjoying the #Olympics and the track cycling starts tomorrow #TeamGB

#Olympics #cyclisme #ciclismo #Radfahren #Wielrennen #kolesarjenje #sykling #cykling #pyöräily #Колоездене… https://t.co/KeeJMHiFnE

Track Cycling Team Pursuit kicks off @ 2:30am EST [2nd August] for 🇨🇦 Women's &amp; Men's Pursuit Teams in Tokyo - here… https://t.co/Ob4lasATlx



# Start creating a data frame

In [28]:
import pandas as pd

In [29]:
def tweet_generator(tweets):
    for tweet in tweets:
        modified_tweet= {}
        
        modified_tweet['user_id']=tweet['user']['id']
        if 'retweeted_status' in tweet:
            modified_tweet['retweeted_status_id']=tweet['retweeted_status']['id']
        else:
            pass
        for i in tweet:
            if i=='retweeted_status' or i=='user':
                pass
            else:
                modified_tweet[i]=tweet[i]
        
        yield modified_tweet  

In [30]:
olympic_cycling_df = pd.DataFrame(tweet_generator(tweets))

In [33]:
olympic_cycling_df.to_pickle("./olympic_cycling_df")

# Check Data Types and Fix Em Up

In [32]:
olympic_cycling_df.dtypes

user_id                        int64
created_at                    object
id                             int64
id_str                        object
text                          object
truncated                       bool
entities                      object
metadata                      object
source                        object
in_reply_to_status_id        float64
in_reply_to_status_id_str     object
in_reply_to_user_id          float64
in_reply_to_user_id_str       object
in_reply_to_screen_name       object
geo                           object
coordinates                   object
place                         object
contributors                  object
is_quote_status                 bool
retweet_count                  int64
favorite_count                 int64
favorited                       bool
retweeted                       bool
lang                          object
possibly_sensitive            object
retweeted_status_id          float64
extended_entities             object
q

# Get replies to tweets:
This link should help with understanding and going through this process:
https://towardsdatascience.com/scraping-specific-tweet-replies-with-python-3143c5214341